In [2]:
from bitarray import bitarray
import sys
import os
SCRIPT_DIR = os.path.dirname(os.path.abspath("/home/canyousee/CLionProjects/PAKE/KC-SPAKE2"))
sys.path.append(os.path.dirname(SCRIPT_DIR))
import json
from binascii import hexlify, unhexlify
from hashlib import sha256
import params
import groups
from params import _Params
from ed25519_group import Ed25519Group
import pandas as pd
from bitarray.util import ba2base, base2ba, ba2int, int2ba
import six
from hashlib import sha256
from itertools import count
from pandas.core.frame import DataFrame
import code
import random
import time
import datetime
import hashlib
from random import randint
from math import floor, log,ceil
from scipy.spatial.distance import hamming
import numpy as np

In [3]:
def popcnt(x):
    return bin(x).count('1')


def dist(x, y):
    return popcnt(x ^ y)


def buildCovering(d, r):
    A = {}
    for v in range(1, 2**(r+1)):
        A[v] = 0
    for i in range(d):
        m = randint(1, 2**(r+1)-1)

        for v in range(1, 2**(r+1)):
            A[v] = A[v] + (1 << i) * (popcnt(m & v) % 2)
    return A


def buildDataStructure(A, S, r):
    D = {}
    for x in S:
        for v in range(1, 2**(r+1)):
            if not (x & A[v]) in D:
                D[x & A[v]] = set()
            D[x & A[v]].add(x)
    return D


def nearestNeighbor(A, D, r, y):
    infinity = float("inf")
    best, nn = infinity, None
    for v in range(1, 2**(r+1)):
        if (y & A[v]) in D:
            for x in D[y & A[v]]:
                if dist(x, y) < best:
                    best, nn = dist(x, y), x

        if best <= floor(log(v+1, 2)):
            return nn
    return None


def generateLSH(A, D, r, y):
    infinity = float("inf")
    result = set()
    best, nn = infinity, None
    for v in range(1, 2**(r+1)):
        if (y & A[v]) in D:
            result.add(y&A[v])
    return result
        


def split_list(a_list, test):
    point = int(len(a_list)*test)
    return a_list[:point], a_list[point:]


In [4]:
df_random=pd.read_csv("/home/canyousee/CLionProjects/CIKM-exper/random.csv")

df_train=pd.read_csv("/home/canyousee/CLionProjects/CIKM-exper/train.csv")

df_plain=pd.read_csv("/home/canyousee/CLionProjects/CIKM-exper/plain.csv")

dff = pd.read_csv(
    "/home/canyousee/CLionProjects/CIKM-exper/Data-profile-source/8000-adult.csv")

In [5]:
df_random=df_random.drop(columns=['order'])
df_train=df_train.drop(columns=['order'])

In [6]:
df_plain=df_plain.drop(columns=['ID'])
df_plain.replace('set()',0,inplace=True)

In [202]:
# bitArr = []
# intArr = []
# ID=[]
# for i in range(0, len(dataSet)):
#     s = bitarray(512)
#     s[:] = 0
#     s[dataSet[i][1]] = 1
#     s[91+dataSet[i][2]] = 1
#     s[101+dataSet[i][3]] = 1
#     s[192+dataSet[i][4]] = 1
#     s[208+dataSet[i][5]] = 1
#     s[215+dataSet[i][6]] = 1
#     s[229+dataSet[i][7]] = 1
#     s[235+dataSet[i][8]] = 1
#     s[240+dataSet[i][9]] = 1
#     s[242+dataSet[i][10]] = 1
#     s[252+dataSet[i][11]] = 1
#     s[256+dataSet[i][12]]=1
#     s[356+dataSet[i][13]]=1
 

#     ID.append(dataSet[i][0])
#     intArr.append(ba2int(s))
#     bitArr.append(s.to01())

# df_bin = pd.DataFrame(ID, columns=['ID'])
# df_bin["int"] = intArr
# df_bin["vector"]=bitArr
# df_bin.to_csv("8000-adult.csv")

In [7]:
dff_set=pd.read_csv("/home/canyousee/CLionProjects/CIKM-exper/Data-profile-vector/8000-adult.csv")

In [8]:
dataSet= []
for i in range(0, dff_set.shape[0]):
    dataSet.append(int(dff_set.iloc[i,2]))

In [205]:
print(dataSet[1])

12786682062094304179739191486143069370074498272330497396351786563005943447724061478367457737875330095156255783665754865259388573693277741016267358208


In [9]:
r = 9 # radius
d = 512  # dimen of vector
   
A =buildCovering(d, r)
D =buildDataStructure(A,dataSet,r)

In [273]:
ff=set()
print(len(ff)==0)

True


In [284]:
df_plain.iloc[0,300]==0

True

In [285]:
tp_all=[]
fp_all=[]
fn_all=[]
tn_all=[]
recall_all=[]
precision_all=[]
di=[255,63,15,1,0]
for p in range(0,6):
    tp=[]
    fp=[]
    fn=[]
    tn=[]
    recall=[]
    precision=[]
    for h in range(0,100):
        if(df_plain.iloc[p,h]!=0):
            testset=[int(x) for x in df_plain.iloc[p,h][1:-1].split(",")]
            tempxs=set(testset)
            tempys=set()
            y1=generateLSH(A, D, r, int(df_random.iloc[h,3]))
            for j in range(0,df_train.shape[0]):
                y2=generateLSH(A,D,r,int(df_train.iloc[j,3]))
                if(len(y2)!=0):
                    y=len(y1&y2)
                    if(y>=di[p]):
                        tempys.add(df_train.iloc[j,2])  
                    
            temp1=6400-len(tempxs|tempys) #tn
            temp2=len(tempxs&tempys) # tp
            temp3=len(tempxs-tempys) # fn
            temp4=len(tempys-tempxs) # fp
            tp.append(temp2)
            fp.append(temp4)
            fn.append(temp3)
            tn.append(temp1)
            #recall.append(temp2/(temp2+temp3))
            #precision.append(temp2/(temp2+temp4))
    tp_all.append(tp)
    fp_all.append(fp)
    fn_all.append(fn)
    tn_all.append(tn)
    #recall_all.append(recall)
    #precision_all.append(precision)

IndexError: list index out of range

In [10]:
tp=[]
fp=[]
fn=[]
tn=[]
recall=[]
precision=[]
di=[255,63,15,15,15,15]
p=3
for h in range(0,50):
    if(df_plain.iloc[p,h]!=0):
        testset=[int(x) for x in df_plain.iloc[p,h][1:-1].split(",")]
        tempxs=set(testset)
        tempys=set()
        y1=generateLSH(A, D, r, int(df_random.iloc[h,3]))
        for j in range(0,df_train.shape[0]):
            y2=generateLSH(A,D,r,int(df_train.iloc[j,3]))
            if(len(y2)!=0):
                y=len(y1&y2)
                if(y>=di[p]):
                    tempys.add(df_train.iloc[j,2])  
                
        temp1=6400-len(tempxs|tempys) #tn
        temp2=len(tempxs&tempys) # tp
        temp3=len(tempxs-tempys) # fn
        temp4=len(tempys-tempxs) # fp
        tp.append(temp2)
        fp.append(temp4)
        fn.append(temp3)
        tn.append(temp1)

In [14]:
for j in range(0,len(tp)):
    print(tp[j],fp[j])

241 10
42 165
555 58
13 8
25 3
44 0
34 62
153 7
732 25
2 3
343 62
106 6
227 4
50 6
24 0
221 24
1 0
198 10
45 8
0 1
0 0
62 5
93 13
7 2
49 7
9 1
64 3
191 47
32 2
68 5
6 0
18 0
15 32
1 0
86 3
216 44
12 41
0 0
17 2
43 3
421 19
21 1
56 2
15 0
93 2
60 2
8 1
42 9


In [15]:
re=[]
pre=[]
for j in range(0,len(tp)):
        tem=tp[j]+fn[j]
        recall=tp[j]/tem
        if(tp[j]==0 and fp[j]==0):
                continue
        tem2= tp[j]+fp[j]
        precision=tp[j]/tem2
        re.append(recall)
        pre.append(precision)

In [16]:
print(re)

[0.4772277227722772, 0.9545454545454546, 0.9053833605220228, 0.20634920634920634, 0.78125, 0.15172413793103448, 0.20238095238095238, 0.4811320754716981, 0.7721518987341772, 0.3333333333333333, 0.6435272045028143, 0.6057142857142858, 0.47789473684210526, 0.4854368932038835, 0.27906976744186046, 0.8215613382899628, 1.0, 0.5577464788732395, 0.45918367346938777, 0.0, 0.30097087378640774, 0.5502958579881657, 0.4117647058823529, 0.7101449275362319, 0.23076923076923078, 0.21476510067114093, 0.8488888888888889, 0.5423728813559322, 0.6666666666666666, 0.15789473684210525, 0.6, 1.0, 0.1111111111111111, 0.7107438016528925, 0.833976833976834, 0.9230769230769231, 0.20481927710843373, 0.22872340425531915, 0.5152998776009792, 0.3181818181818182, 0.3128491620111732, 0.15, 0.23425692695214106, 0.3508771929824561, 0.23529411764705882, 0.20388349514563106]


In [19]:
a=pre
b=np.var(a)
c=np.mean(a)
print(b)
print(c)

0.05818203038931988
0.8331241808330685


In [12]:
i=1
j=35
print(tp_all[i][j])
tem2= tp_all[i][j]+fp_all[i][j]

if tp_all[i][j]/tem2==0:
        print(j,tem2)


NameError: name 'tp_all' is not defined

In [330]:
recall_all=[]
precision_all=[]
for i in range(0,5):
    al=len(tp_all[0])
    re=[]
    pre=[]
    for j in range(0,len(tp_all[i])):
        tem=tp_all[i][j]+fn_all[i][j]
        recall=tp_all[i][j]/tem
        tem2= tp_all[i][j]+fp_all[i][j]
        precision=tp_all[i][j]/tem2
        re.append(recall)
        pre.append(pre)
    recall_all.append(re)
    precision_all.append(pre)

In [331]:
print(recall_all)
print(precision_all)

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 0.8, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9555555555555556, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.

In [264]:
testset=[int(x) for x in df_plain.iloc[0,1599][1:-1].split(",")]
testset=set(testset)
print(testset)

{25576}


In [265]:
result=set()
k=1599
print(df_random.iloc[k,2])
y1=generateLSH(A, D, r, int(df_random.iloc[k,3]))
for j in range(0,df_train.shape[0]):
    y2=generateLSH(A,D,r,int(df_train.iloc[j,3]))
    y=len(y1&y2)
    if(y>=1):
        result.add(df_train.iloc[j,2])  

17297


In [266]:
id_hash=[]
for id in testset:
    y2=generateLSH(A,D,r,int(df_train[df_train['ID']==id].iloc[0,3]))

    id_hash.append(len(y1&y2))
print(id_hash)
print(testset&result==testset)

[255]
True


In [ ]:
random_test_lsh_results=[]
count=0
for k in range(0,df_random.shape[0]):
    result=set()
    y1=generateLSH(A, D, r, int(df_random.iloc[k,3]))
    
    for j in range(0,df_train.shape[0]):
            y2=generateLSH(A,D,r,int(df_train.iloc[j,3]))
            y=len(y1&y2)
            if(y>=50):
                result.add(df_train.iloc[j,2])               
    random_test_lsh_results.append(result)


In [115]:
result_all_lsh=[]
for p in range(1,13):
    random_test_lsh_results=[]
    for k in range(0,df_random.shape[0]):
        result=set()
        y1=generateLSH(A, D, r, int(df_random.iloc[k,3]))
        for j in range(0,df_train.shape[0]):
                y2=generateLSH(A,D,r,int(df_train.iloc[j,3]))
                y=len(y1&y2)
                if(y>):
                  result.add(df_train.iloc[j,3])      
        random_test_lsh_results.append(result)
    result_all_lsh.append(random_test_lsh_results)

In [157]:
result_alllsh_pd=pd.DataFrame(result_all_lsh)
result_alllsh_pd.to_csv("lsh_withalldata.csv")

In [163]:
df_all_lsh=pd.read_csv("lsh_withalldata.csv")

In [141]:
result_all_lsh=[]
for p in range(1,13):
    random_test_lsh_results=[]
    for k in range(0,df_random.shape[0]):
        result=set()
        y1=generateLSH(A, D, r, int(df_random.iloc[k,3]))
        for j in range(0,df_set.shape[0]):
                y2=generateLSH(A,D,r,int(df_set.iloc[j,2]))
                if(y1==y2):
                  result.add(df_set.iloc[j,1])      
        random_test_lsh_results.append(result)
    result_all_lsh.append(random_test_lsh_results)

In [142]:
result_alllsh_pd=pd.DataFrame(result_all_lsh)
result_alllsh_pd.to_csv("lsh_withalldata_all.csv")

In [143]:
df_all_lsh_all=pd.read_csv("lsh_withalldata_all.csv")

In [11]:
k=0
p=3
print(df_random.iloc[k,])
plaintemp=dff[dff['ID']==df_random.iloc[k,2]].iloc[0,1:]
result=set()
for i in range(0,df_train.shape[0]):
        plaintemp2=dff[dff['ID']==df_train.iloc[i,2]].iloc[0,1:]
        disti=hamming(plaintemp,plaintemp2)*13
        if(disti<=p):
            result.add(df_train.iloc[i,2])   


level_0                                                 5788
index                                                   5788
ID                                                     17009
int        6393341031047152089869680356944416129127154323...
vector     0000000000000000000010000000000000000000000000...
Name: 0, dtype: object


In [20]:
result=set()
y1=generateLSH(A, D, r, int(df_random.iloc[k,3]))
for j in range(0,df_train.shape[0]):
        y2=generateLSH(A,D,r,int(df_train.iloc[j,3]))
        if(y1==y2):
            result.add(df_train.iloc[j,2])      


In [21]:
print(result)

{13011}


In [140]:
result_all_lsh=[]
for p in range(1,13):
    random_test_lsh_results=[]
    for k in range(0,df_random.shape[0]):
        result=set()
        y1=generateLSH(A, D, r, int(df_random.iloc[k,3]))
        for j in range(0,df_train.shape[0]):
                y2=generateLSH(A,D,r,int(df_train.iloc[j,3]))
                if(y1==y2):
                  result.add(df_train.iloc[j,2])      
        random_test_lsh_results.append(result)
    result_all_lsh.append(random_test_lsh_results)

In [141]:
result_alllsh_pd=pd.DataFrame(result_all_lsh)
result_alllsh_pd.to_csv("lsh.csv")

In [6]:
result_alllsh_pd=pd.read_csv("lsh.csv")


In [7]:
result_alllsh_pd=result_alllsh_pd.drop(columns=['ID'])
df_plain=df_plain.drop(columns=['ID'])
df_plain.replace('set()',0,inplace=True)

In [8]:
df_plain=df_plain.drop(columns=['ID'])

In [11]:
df_plain.replace('set()',0,inplace=True)
result_alllsh_pd.replace('set()',0,inplace=True)

In [13]:

recall_all=[]
precision_all=[]

for p in range(0,12):
    tp=[]
    fp=[]
    fn=[]
    tn=[]
    recall=[]
    precision=[]
    for i in range(0,df_random.shape[0]):
        if(df_plain.iloc[p,i]!=0 and result_alllsh_pd.iloc[p,i]!=0):
            tempx=[int(x) for x in df_plain.iloc[p,i][1:-1].split(",")]
            tempxs=set(tempx)

            tempy=[int(x) for x in result_alllsh_pd.iloc[p,i][1:-1].split(",")]
            tempys=set(tempy)

            #print("plain",tempxs)
            #print("result",tempys)
            temp1=6400-len(tempxs|tempys) #tn
            temp2=len(tempxs&tempys) # tp
            temp3=len(tempxs-tempys) # fn
            temp4=len(tempys-tempxs) # fp
            tp.append(temp2)
            fp.append(temp4)
            fn.append(temp3)
            tn.append(temp1)
            recall.append(temp2/(temp2+temp3))
            precision.append(temp2/(temp2+temp4))
    recall_all.append(recall)
    precision_all.append(precision)





In [22]:
ba=[]
ca=[]
for i in range(0,12):
    a=np.array(recall_all[i])
    b=np.var(a)
    c=np.mean(a)
    ba.append(b)
    ca.append(c)
    print(b,c)

0.162761466352013 0.6727887554500458
0.1033614928839701 0.3642350634138221
0.02992594702075325 0.14601769462890127
0.006370014953143769 0.05689370334170711
0.0009915124849907107 0.022016969864820372
0.00015373448992720331 0.009218208460740148
5.217891916161544e-05 0.005324022044015981
2.7327712382135196e-05 0.0037169659400743443
2.0012933624069718e-05 0.003120236004037052
1.829677406612699e-05 0.002961313840532364
1.803501637568629e-05 0.002934849743299574
1.8014121764301052e-05 0.0029327200075166754


In [26]:
recall_pd=pd.DataFrame(ba,columns=['recall_var'])
recall_pd['recall_mean']=ca
print(recall_pd)
recall_pd.to_csv("recall.csv")

    recall_var  recall_mean
0     0.162761     0.672789
1     0.103361     0.364235
2     0.029926     0.146018
3     0.006370     0.056894
4     0.000992     0.022017
5     0.000154     0.009218
6     0.000052     0.005324
7     0.000027     0.003717
8     0.000020     0.003120
9     0.000018     0.002961
10    0.000018     0.002935
11    0.000018     0.002933


In [27]:
ba=[]
ca=[]
for i in range(0,12):
    a=np.array(precision_all[i])
    b=np.var(a)
    c=np.mean(a)
    ba.append(b)
    ca.append(c)
    print(b,c)

0.11371893531456685 0.22912142326680213
0.12364105608997221 0.32474229849055786
0.14191381711680276 0.5176757979989016
0.10164254173667067 0.7620205367039936
0.031816756746597084 0.9267787133090322
0.009272844945225171 0.9800115701716805
0.001402744885307406 0.9971614527331457
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0


In [28]:
precision_all_pd=pd.DataFrame(ba,columns=['precision_var'])
precision_all_pd['precision_mean']=ca
print(precision_all_pd)
precision_all_pd.to_csv("precision.csv")

    precision_var  precision_mean
0        0.113719        0.229121
1        0.123641        0.324742
2        0.141914        0.517676
3        0.101643        0.762021
4        0.031817        0.926779
5        0.009273        0.980012
6        0.001403        0.997161
7        0.000000        1.000000
8        0.000000        1.000000
9        0.000000        1.000000
10       0.000000        1.000000
11       0.000000        1.000000
